In [1]:
import numpy as np
import math

In [2]:
class Apriori:
    def __init__(self):
        self.k=0#不知道填啥，就随便写了一个变量，没啥用，不用关心
        
    def createC1(self,dataset):#构建一个项为1的频繁项集,即将所有出现的项放入一个列表中
        C1=[]
        for dataline in datasets:
            for item in dataline:
                if [item] not in C1:
                    C1.append([item])
        C1.sort()
        return list(map(frozenset,C1))#将C1集合冰冻，让其变成一个不可修改的集合,再将其转化为一个列表，方便循环
    
    def scanD(self,datasets,C1,minsupport):#数据集，频繁项集，最低支持度
        sscnt={}#频繁项集及其对应的支持度
        for tid in datasets:
            for can in C1:
                if can.issubset(tid):
                    if can not in sscnt.keys():
                        sscnt[can]=1
                    else:
                        sscnt[can]+=1
        totalnum=len(datasets)#接下来计算支持度
        retlist=[]
        supportdata={}#存放项集以及其支持度
        for key in sscnt.keys():
            support=sscnt[key]/totalnum
            if support>minsupport:
                retlist.append(key)#返回支持度大于设定的支持度的频繁项集
            supportdata[key]=support
        return retlist,supportdata
        
    def apriorigen(self,lk):#此函数主要为产生接下来的更多项集
        retlist=[]
        lenlk=len(lk)
        for i in range(lenlk):
            for j in range(i+1,lenlk):
                L1=lk[i]#将第i项中的集合元素取出来
                L2=lk[j]#将第j项中的集合元素取出来
                retlist.append(L1|L2)#计算L1和L2的并集，并将其添加到列表中
        retlist1=list(set(retlist))#每一次运行合并后的频繁项集不能有重复的，因此需要对已经分好的频繁项集去一下重
        return retlist1
    
    def apriori(self,datasets,minsupport=0.3):#输入数据集和最低支持度，直接输出多项结果，L[0]为一项集，L[1]为二项集，以此类推        
        C1=self.createC1(datasets)
        Lk,supportdata=self.scanD(datasets,C1,minsupport)#生成一项集
        L=[Lk]
        k=0
        while (len(L[k])>0):#循环执行，直到不存在频繁项集为止
            Ck=self.apriorigen(Lk)#生成二项集
            Lk,supk=self.scanD(datasets,Ck,minsupport)#生成二项集下的集合和支持度
            supportdata.update(supk)
            L.append(Lk)
            k+=1
        return L,supportdata
    
    #以下为关联规则的函数
    def calcondf(self,freqset,supportdata,H1,mincondf=0.5):#计算置信度
        cal=[]
        for H in H1:
            condf=supportdata[freqset]/supportdata[freqset-H]
            if condf>=mincondf:
                cal.append((freqset-H,H,condf))
                print(freqset-H,'-->',H,'可信度为：',condf)
        return cal 
    
    def rulegen(self,datasets,minsupport=0.3,mincondf=0.5):
        L,supportdata=self.apriori(datasets,minsupport)
        bigru=[]
        for i in range(len(L)):
            for freqset in L[i]:
                H1=[frozenset([item]) for item in freqset]
                if i>0:
                    km=self.calcondf(freqset,supportdata,H1,mincondf)
                    bigru.append(km)
        return bigru

In [3]:
datasets=[[1,3,4],[2,3,5],[1,2,3,5],[2,5]]

In [4]:
ap=Apriori()
C1=ap.createC1(datasets)
L=ap.rulegen(datasets,minsupport=0.3,mincondf=0.5)
L

frozenset({3}) --> frozenset({1}) 可信度为： 0.6666666666666666
frozenset({1}) --> frozenset({3}) 可信度为： 1.0
frozenset({3}) --> frozenset({2}) 可信度为： 0.6666666666666666
frozenset({2}) --> frozenset({3}) 可信度为： 0.6666666666666666
frozenset({5}) --> frozenset({2}) 可信度为： 1.0
frozenset({2}) --> frozenset({5}) 可信度为： 1.0
frozenset({5}) --> frozenset({3}) 可信度为： 0.6666666666666666
frozenset({3}) --> frozenset({5}) 可信度为： 0.6666666666666666
frozenset({3, 5}) --> frozenset({2}) 可信度为： 1.0
frozenset({2, 5}) --> frozenset({3}) 可信度为： 0.6666666666666666
frozenset({2, 3}) --> frozenset({5}) 可信度为： 1.0


[[(frozenset({3}), frozenset({1}), 0.6666666666666666),
  (frozenset({1}), frozenset({3}), 1.0)],
 [(frozenset({3}), frozenset({2}), 0.6666666666666666),
  (frozenset({2}), frozenset({3}), 0.6666666666666666)],
 [(frozenset({5}), frozenset({2}), 1.0),
  (frozenset({2}), frozenset({5}), 1.0)],
 [(frozenset({5}), frozenset({3}), 0.6666666666666666),
  (frozenset({3}), frozenset({5}), 0.6666666666666666)],
 [(frozenset({3, 5}), frozenset({2}), 1.0),
  (frozenset({2, 5}), frozenset({3}), 0.6666666666666666),
  (frozenset({2, 3}), frozenset({5}), 1.0)]]